# The following script is derived from the following resource:  
### https://github.com/ESA-PhiLab/radiometric-slope-correction
 RTC correction below is entirely copied from the above resource until step 4.  Step 4 is modified from the original resource to meet the needs of the current study by limiting the bands output to the users drive, limiting the output to the dat range fomr the study, and by modifying the paths and titles of the output resource to include the dates of the imagery.  This script may be run from a blank virtual environment.  All the installation is included in the first executable cell in the notebook. 

# 1 - Get connected to Google Earth Engine

In [ ]:
!pip3 install google-api-python-client google-auth-httplib2 google-auth-oauthlib tabulate requests tqdm
!pip install numpy

In [1]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

Enter verification code: 4/1AY0e-g4FnHA25rcSU3JZm2KyoBmDc5KqDv6Y5t6R-Sw25KBShJor_G1ujFc

Successfully saved authorization token.


# 2 - Import necessary python libs

In [2]:
import numpy as np
ee.Initialize()

# 3 - The Model function

This is a nested function, so that we can actually map over a collection, by simultaneously giving the DEM, model of choice and buffer parameter.

In [3]:
def slope_correction(collection, elevation, model, buffer=0):
    '''This function applies the slope correction on a collection of Sentinel-1 data
       
       :param collection: ee.Collection of Sentinel-1
       :param elevation: ee.Image of DEM
       :param model: model to be applied (volume/surface)
       :param buffer: buffer in meters for layover/shadow amsk
        
        :returns: ee.Image
    '''
    
    def _volumetric_model_SCF(theta_iRad, alpha_rRad):
        '''Code for calculation of volumetric model SCF
        
        :param theta_iRad: ee.Image of incidence angle in radians
        :param alpha_rRad: ee.Image of slope steepness in range
        
        :returns: ee.Image
        '''
        
        # create a 90 degree image in radians
        ninetyRad = ee.Image.constant(90).multiply(np.pi/180)
        
        # model
        nominator = (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).tan()
        denominator = (ninetyRad.subtract(theta_iRad)).tan()
        return nominator.divide(denominator) 
    
    
    def _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad):
        '''Code for calculation of direct model SCF
        
        :param theta_iRad: ee.Image of incidence angle in radians
        :param alpha_rRad: ee.Image of slope steepness in range
        :param alpha_azRad: ee.Image of slope steepness in azimuth
        
        :returns: ee.Image
        '''
        
        # create a 90 degree image in radians
        ninetyRad = ee.Image.constant(90).multiply(np.pi/180)
        
        # model  
        nominator = (ninetyRad.subtract(theta_iRad)).cos()
        denominator = (alpha_azRad.cos()
          .multiply((ninetyRad.subtract(theta_iRad).add(alpha_rRad)).cos()))

        return nominator.divide(denominator)


    def _erode(image, distance):
        '''Buffer function for raster

        :param image: ee.Image that shoudl be buffered
        :param distance: distance of buffer in meters
        
        :returns: ee.Image
        '''
      
        d = (image.Not().unmask(1)
          .fastDistanceTransform(30).sqrt()
          .multiply(ee.Image.pixelArea().sqrt()))
    
        return image.updateMask(d.gt(distance))
    
    
    def _masking(alpha_rRad, theta_iRad, buffer):
        '''Masking of layover and shadow
        
        
        :param alpha_rRad: ee.Image of slope steepness in range
        :param theta_iRad: ee.Image of incidence angle in radians
        :param buffer: buffer in meters
        
        :returns: ee.Image
        '''
        # layover, where slope > radar viewing angle 
        layover = alpha_rRad.lt(theta_iRad).rename('layover')

        # shadow 
        ninetyRad = ee.Image.constant(90).multiply(np.pi/180)
        shadow = alpha_rRad.gt(ee.Image.constant(-1).multiply(ninetyRad.subtract(theta_iRad))).rename('shadow')
        
        # add buffer to layover and shadow
        if buffer > 0:
            layover = _erode(layover, buffer)   
            shadow = _erode(shadow, buffer)  

        # combine layover and shadow
        no_data_mask = layover.And(shadow).rename('no_data_mask')
        
        return layover.addBands(shadow).addBands(no_data_mask)
                        
        
    def _correct(image):
        '''This function applies the slope correction and adds layover and shadow masks
        
        '''
        
        # get the image geometry and projection
        geom = image.geometry()
        proj = image.select(1).projection()
        
        # calculate the look direction
        heading = (ee.Terrain.aspect(image.select('angle'))
                                     .reduceRegion(ee.Reducer.mean(), geom, 1000)
                                     .get('aspect'))
                   

        # Sigma0 to Power of input image
        sigma0Pow = ee.Image.constant(10).pow(image.divide(10.0))

        # the numbering follows the article chapters
        # 2.1.1 Radar geometry 
        theta_iRad = image.select('angle').multiply(np.pi/180)
        phi_iRad = ee.Image.constant(heading).multiply(np.pi/180)
        
        # 2.1.2 Terrain geometry
        alpha_sRad = ee.Terrain.slope(elevation).select('slope').multiply(np.pi/180).setDefaultProjection(proj).clip(geom)
        phi_sRad = ee.Terrain.aspect(elevation).select('aspect').multiply(np.pi/180).setDefaultProjection(proj).clip(geom)
        
        # we get the height, for export 
        height = elevation.setDefaultProjection(proj).clip(geom)
        
        # 2.1.3 Model geometry
        #reduce to 3 angle
        phi_rRad = phi_iRad.subtract(phi_sRad)

        # slope steepness in range (eq. 2)
        alpha_rRad = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()

        # slope steepness in azimuth (eq 3)
        alpha_azRad = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()

        # local incidence angle (eq. 4)
        theta_liaRad = (alpha_azRad.cos().multiply((theta_iRad.subtract(alpha_rRad)).cos())).acos()
        theta_liaDeg = theta_liaRad.multiply(180/np.pi)

        # 2.2 
        # Gamma_nought
        gamma0 = sigma0Pow.divide(theta_iRad.cos())
        gamma0dB = ee.Image.constant(10).multiply(gamma0.log10()).select(['VV', 'VH'], ['VV_gamma0', 'VH_gamma0'])
        ratio_gamma = (gamma0dB.select('VV_gamma0')
                        .subtract(gamma0dB.select('VH_gamma0'))
                        .rename('ratio_gamma0'))

        if model == 'volume':
            scf = _volumetric_model_SCF(theta_iRad, alpha_rRad)

        if model == 'surface':
            scf = _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad)

        # apply model for Gamm0_f
        gamma0_flat = gamma0.divide(scf)
        gamma0_flatDB = (ee.Image.constant(10)
                         .multiply(gamma0_flat.log10())
                         .select(['VV', 'VH'],['VV_gamma0flat', 'VH_gamma0flat'])
                        )

        masks = _masking(alpha_rRad, theta_iRad, buffer)

        # calculate the ratio for RGB vis
        ratio_flat = (gamma0_flatDB.select('VV_gamma0flat')
                        .subtract(gamma0_flatDB.select('VH_gamma0flat'))
                        .rename('ratio_gamma0flat')
                     )

        return (image.rename(['VV_sigma0', 'VH_sigma0', 'incAngle'])
                      .addBands(gamma0dB)
                      .addBands(ratio_gamma)
                      .addBands(gamma0_flatDB)
                      .addBands(ratio_flat)
                      .addBands(alpha_rRad.rename('alpha_rRad'))
                      .addBands(alpha_azRad.rename('alpha_azRad'))
                      .addBands(phi_sRad.rename('aspect'))
                      .addBands(alpha_sRad.rename('slope'))
                      .addBands(theta_iRad.rename('theta_iRad'))
                      .addBands(theta_liaRad.rename('theta_liaRad'))
                      .addBands(masks)
                      .addBands(height.rename('elevation'))
                 )    
    
    # run and return correction
    return collection.map(_correct)

# 3 - Create corrected data for volume and surface model on Earth Engine and export both to Google Drive

Within this code, we apply the slope correction function to two consecutive frames of Sentinel-1 using the volume and surface model seaprately. All layers are exported to Google Drive.

In [5]:
import datetime



date_ranges = [[datetime.datetime(2015, 12, 1), datetime.datetime(2016, 5, 1)],# date ranges of interest
    [datetime.datetime(2016, 12, 1), datetime.datetime(2017, 5, 1)],
    [datetime.datetime(2018, 12, 1), datetime.datetime(2019, 5, 1)]]


coords = [[[19.73674389730968,69.606875618026777], # AOI near Tromsoe
            [20.285778371170068,69.588085050933046],
            [20.23001507315962,69.396611998021626],
            [19.68581232015714,69.415216343329689],
            [19.73674389730968,69.606875618026777]]]

geometry = ee.Geometry.Polygon(coords)

paths = [66, 131, 95, 160] # S-1 paths to collect geometry from  

for date in date_ranges:
    for path in paths:
        start = date[0]
        while start <= date[1]:
            stop = start + datetime.timedelta(days=1)# check next day between date ranges
            try:
                # filter Sentinel-1 collection for AOI and selected dates
                s1Collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
                    .filterBounds(geometry) \
                    .filterDate(start.strftime("%F"), stop.strftime("%F"))

                if s1Collection.first().getInfo()['properties']['relativeOrbitNumber_stop'] == path:
                    
                    # paths to dem on earth engine 
                    dem = 'users/davidharr90/S1_Tromsoe_DEM'# DEM for AOI previously added to GEE

                    # this is the scale we want the data to be sampled
                    scale = 10 

                    # get the respective collection and bands and mosaic to a single image
                    corrected_image = slope_correction(
                        s1Collection, 
                        ee.Image(dem), 
                        'surface' # restrict the output to only the surface model
                    ).mosaic()
        
                    # we get geometry and projection of the image
                    proj = corrected_image.select(1).projection()
                    geom = corrected_image.clip(geometry).select(1).geometry()
        
                    image_reprojected = corrected_image.reproject(proj, scale=scale).clip(geom)
      
                    # get the bandlist 
                    bandlist = image_reprojected.bandNames().getInfo()
    
                    desired_bands = ['VH_gamma0', 'VV_gamma0', 'VH_sigma0', 'VV_sigma0', 'layover', 'shadow']
    
                    # create an export job for each band
                    for band in bandlist:
                        if band in desired_bands:
                            task = ee.batch.Export.image.toDrive(
                                image=image_reprojected.select(band).clip(geom),
                                description=band,
                                folder='{}_{}_buf_0_date_{}'.format(dem.split('/')[-1], 
                                                                'surface', start.strftime("%F")),
                                fileNamePrefix='{}_{}_{}'.format(band, start.strftime("%F"), path),
                                region=geom.coordinates().getInfo(),
                                scale=scale,
                                maxPixels=1e12
                            )
                            task.start()
                
                            print(task.status())
                start = stop
            except Exception as E:
                print('no imagery available on {}'.format(start.strftime("%F")))
                start = stop

no imagery available on 2015-12-01
no imagery available on 2015-12-02
no imagery available on 2015-12-03
VV_sigma0 2015-12-04 00:00:00
{'state': 'READY', 'description': 'VV_sigma0', 'creation_timestamp_ms': 1617477228182, 'update_timestamp_ms': 1617477228182, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'QP6N5D2ACDHTIA3FJJ7DKG2D', 'name': 'projects/earthengine-legacy/operations/QP6N5D2ACDHTIA3FJJ7DKG2D'}
VH_sigma0 2015-12-04 00:00:00
{'state': 'READY', 'description': 'VH_sigma0', 'creation_timestamp_ms': 1617477228920, 'update_timestamp_ms': 1617477228920, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'YZNHZYVWIPQKSIBJJTMPFYID', 'name': 'projects/earthengine-legacy/operations/YZNHZYVWIPQKSIBJJTMPFYID'}
VV_gamma0 2015-12-04 00:00:00
{'state': 'READY', 'description': 'VV_gamma0', 'creation_timestamp_ms': 1617477229647, 'update_timestamp_ms': 1617477229647, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'CRUSJXNS6II5GUYO2ACTGTZ5', 'name': 'project

no imagery available on 2016-01-14
no imagery available on 2016-01-17
no imagery available on 2016-01-18
no imagery available on 2016-01-19
no imagery available on 2016-01-20
VV_sigma0 2016-01-21 00:00:00
{'state': 'READY', 'description': 'VV_sigma0', 'creation_timestamp_ms': 1617477254002, 'update_timestamp_ms': 1617477254002, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'AUEQFXQCMOTMBJG5AX6GQF7V', 'name': 'projects/earthengine-legacy/operations/AUEQFXQCMOTMBJG5AX6GQF7V'}
VH_sigma0 2016-01-21 00:00:00
{'state': 'READY', 'description': 'VH_sigma0', 'creation_timestamp_ms': 1617477254718, 'update_timestamp_ms': 1617477254718, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'P6XOAQTDNSYVNLSJMWYGXSSG', 'name': 'projects/earthengine-legacy/operations/P6XOAQTDNSYVNLSJMWYGXSSG'}
VV_gamma0 2016-01-21 00:00:00
{'state': 'READY', 'description': 'VV_gamma0', 'creation_timestamp_ms': 1617477255377, 'update_timestamp_ms': 1617477255377, 'start_timestamp_ms': 0, 'task_typ

no imagery available on 2016-03-02
no imagery available on 2016-03-05
no imagery available on 2016-03-06
no imagery available on 2016-03-07
no imagery available on 2016-03-08
VV_sigma0 2016-03-09 00:00:00
{'state': 'READY', 'description': 'VV_sigma0', 'creation_timestamp_ms': 1617477278995, 'update_timestamp_ms': 1617477278995, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'RCI3J4VATSV5SD36HQ4YVCWZ', 'name': 'projects/earthengine-legacy/operations/RCI3J4VATSV5SD36HQ4YVCWZ'}
VH_sigma0 2016-03-09 00:00:00
{'state': 'READY', 'description': 'VH_sigma0', 'creation_timestamp_ms': 1617477279750, 'update_timestamp_ms': 1617477279750, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'KDVE623OGYPNRBNIW43BUBRL', 'name': 'projects/earthengine-legacy/operations/KDVE623OGYPNRBNIW43BUBRL'}
VV_gamma0 2016-03-09 00:00:00
{'state': 'READY', 'description': 'VV_gamma0', 'creation_timestamp_ms': 1617477280416, 'update_timestamp_ms': 1617477280416, 'start_timestamp_ms': 0, 'task_typ

no imagery available on 2016-04-19
no imagery available on 2016-04-22
no imagery available on 2016-04-23
no imagery available on 2016-04-24
no imagery available on 2016-04-25
VV_sigma0 2016-04-26 00:00:00
{'state': 'READY', 'description': 'VV_sigma0', 'creation_timestamp_ms': 1617477304022, 'update_timestamp_ms': 1617477304022, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'OC2YGQFBJBELMXTD7RFAPXL6', 'name': 'projects/earthengine-legacy/operations/OC2YGQFBJBELMXTD7RFAPXL6'}
VH_sigma0 2016-04-26 00:00:00
{'state': 'READY', 'description': 'VH_sigma0', 'creation_timestamp_ms': 1617477304802, 'update_timestamp_ms': 1617477304802, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'TKZ3YAGE7UPE2UPEM6AKFDKD', 'name': 'projects/earthengine-legacy/operations/TKZ3YAGE7UPE2UPEM6AKFDKD'}
VV_gamma0 2016-04-26 00:00:00
{'state': 'READY', 'description': 'VV_gamma0', 'creation_timestamp_ms': 1617477305499, 'update_timestamp_ms': 1617477305499, 'start_timestamp_ms': 0, 'task_typ

no imagery available on 2016-01-05
no imagery available on 2016-01-06
no imagery available on 2016-01-07
no imagery available on 2016-01-08
VV_sigma0 2016-01-13 00:00:00
{'state': 'READY', 'description': 'VV_sigma0', 'creation_timestamp_ms': 1617477328728, 'update_timestamp_ms': 1617477328728, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'AQAJTBYPNAHKHN6RJ6PFSTCO', 'name': 'projects/earthengine-legacy/operations/AQAJTBYPNAHKHN6RJ6PFSTCO'}
VH_sigma0 2016-01-13 00:00:00
{'state': 'READY', 'description': 'VH_sigma0', 'creation_timestamp_ms': 1617477329539, 'update_timestamp_ms': 1617477329539, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'KBKMOWISTZW5HJJNESPLRN7U', 'name': 'projects/earthengine-legacy/operations/KBKMOWISTZW5HJJNESPLRN7U'}
VV_gamma0 2016-01-13 00:00:00
{'state': 'READY', 'description': 'VV_gamma0', 'creation_timestamp_ms': 1617477330220, 'update_timestamp_ms': 1617477330220, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'SO4JUQ5F

...

{'state': 'READY', 'description': 'VH_sigma0', 'creation_timestamp_ms': 1617478350876, 'update_timestamp_ms': 1617478350876, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'GYKBMMUR4ORUOKF5ZD7A2S3M', 'name': 'projects/earthengine-legacy/operations/GYKBMMUR4ORUOKF5ZD7A2S3M'}
VV_gamma0 2019-03-26 00:00:00
{'state': 'READY', 'description': 'VV_gamma0', 'creation_timestamp_ms': 1617478351694, 'update_timestamp_ms': 1617478351694, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'JJV2DGONVKFIZKV6SYNSLY4J', 'name': 'projects/earthengine-legacy/operations/JJV2DGONVKFIZKV6SYNSLY4J'}
VH_gamma0 2019-03-26 00:00:00
{'state': 'READY', 'description': 'VH_gamma0', 'creation_timestamp_ms': 1617478352381, 'update_timestamp_ms': 1617478352381, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'QS7Y5PMAQ37UJVP55V655RCU', 'name': 'projects/earthengine-legacy/operations/QS7Y5PMAQ37UJVP55V655RCU'}
layover 2019-03-26 00:00:00
{'state': 'READY', 'description': 'layover', 'cr

{'state': 'READY', 'description': 'VH_gamma0', 'creation_timestamp_ms': 1617478374404, 'update_timestamp_ms': 1617478374404, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'JM4EBEL6CAKQPBU4QCFII5MJ', 'name': 'projects/earthengine-legacy/operations/JM4EBEL6CAKQPBU4QCFII5MJ'}
layover 2019-04-19 00:00:00
{'state': 'READY', 'description': 'layover', 'creation_timestamp_ms': 1617478375156, 'update_timestamp_ms': 1617478375156, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'DBALF6FS5WG4B6RA6EGG7ODN', 'name': 'projects/earthengine-legacy/operations/DBALF6FS5WG4B6RA6EGG7ODN'}
shadow 2019-04-19 00:00:00
{'state': 'READY', 'description': 'shadow', 'creation_timestamp_ms': 1617478375822, 'update_timestamp_ms': 1617478375822, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '6WNPV7F7CXGSTHN6AE2DJCJZ', 'name': 'projects/earthengine-legacy/operations/6WNPV7F7CXGSTHN6AE2DJCJZ'}
VV_sigma0 2019-04-25 00:00:00
{'state': 'READY', 'description': 'VV_sigma0', 'creation

{'state': 'READY', 'description': 'shadow', 'creation_timestamp_ms': 1617478399170, 'update_timestamp_ms': 1617478399170, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'VO2GOIQI5B5SA2W7LF7NCQY5', 'name': 'projects/earthengine-legacy/operations/VO2GOIQI5B5SA2W7LF7NCQY5'}
VV_sigma0 2018-12-30 00:00:00
{'state': 'READY', 'description': 'VV_sigma0', 'creation_timestamp_ms': 1617478401590, 'update_timestamp_ms': 1617478401590, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'AJGHTJOXCM64XGGLE24VUX4R', 'name': 'projects/earthengine-legacy/operations/AJGHTJOXCM64XGGLE24VUX4R'}
VH_sigma0 2018-12-30 00:00:00
{'state': 'READY', 'description': 'VH_sigma0', 'creation_timestamp_ms': 1617478402734, 'update_timestamp_ms': 1617478402734, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'JBH5ZENUT6FUYYQPANAO3INP', 'name': 'projects/earthengine-legacy/operations/JBH5ZENUT6FUYYQPANAO3INP'}
VV_gamma0 2018-12-30 00:00:00
{'state': 'READY', 'description': 'VV_gamma0', 'c

{'state': 'READY', 'description': 'VH_sigma0', 'creation_timestamp_ms': 1617478428721, 'update_timestamp_ms': 1617478428721, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'SGBV6I3TZUYGNUNAKUZKYSLC', 'name': 'projects/earthengine-legacy/operations/SGBV6I3TZUYGNUNAKUZKYSLC'}
VV_gamma0 2019-02-16 00:00:00
{'state': 'READY', 'description': 'VV_gamma0', 'creation_timestamp_ms': 1617478429402, 'update_timestamp_ms': 1617478429402, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'FEHSFQQKEUEDFB57EJUQ3PHK', 'name': 'projects/earthengine-legacy/operations/FEHSFQQKEUEDFB57EJUQ3PHK'}
VH_gamma0 2019-02-16 00:00:00
{'state': 'READY', 'description': 'VH_gamma0', 'creation_timestamp_ms': 1617478430174, 'update_timestamp_ms': 1617478430174, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'SSUKME5HS5UCD6Y7OISUOPEB', 'name': 'projects/earthengine-legacy/operations/SSUKME5HS5UCD6Y7OISUOPEB'}
layover 2019-02-16 00:00:00
{'state': 'READY', 'description': 'layover', 'cr

{'state': 'READY', 'description': 'VH_gamma0', 'creation_timestamp_ms': 1617478456857, 'update_timestamp_ms': 1617478456857, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '4JO4PXDBMVS7CI5HBVBKFA4B', 'name': 'projects/earthengine-legacy/operations/4JO4PXDBMVS7CI5HBVBKFA4B'}
layover 2019-04-05 00:00:00
{'state': 'READY', 'description': 'layover', 'creation_timestamp_ms': 1617478457496, 'update_timestamp_ms': 1617478457496, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'DRI4JPIUPSZYMMVH7YBQIV6D', 'name': 'projects/earthengine-legacy/operations/DRI4JPIUPSZYMMVH7YBQIV6D'}
shadow 2019-04-05 00:00:00
{'state': 'READY', 'description': 'shadow', 'creation_timestamp_ms': 1617478458328, 'update_timestamp_ms': 1617478458328, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'DJ3WKT3BQ7VPBQOF4QAKQ6TB', 'name': 'projects/earthengine-legacy/operations/DJ3WKT3BQ7VPBQOF4QAKQ6TB'}
VV_sigma0 2019-04-17 00:00:00
{'state': 'READY', 'description': 'VV_sigma0', 'creation